## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 9th Mar. 16th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,March-11-2025,Condo/Co-op,6550 S Oriole Blvd Unit 102 I,Delray Beach,FL,33446.0,135000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/6550-S-...,Beaches MLS,RX-11070434,N,Y,26.456076,-80.156025


In [8]:
print('Input county name:')
county = input()

Input county name:
Palm Beach County


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-03-10    42
2025-03-14    38
2025-03-12    21
2025-03-11    19
2025-03-13    18
2025-03-15     1
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [15]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
132,PAST SALE,2025-03-10,Condo/Co-op,1100 S Flagler Dr #1401,West Palm Beach,FL,33401.0,14900000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-10972146,N,Y,26.703917,-80.049385
33,PAST SALE,2025-03-12,Condo/Co-op,100 Sunrise Ave Ph 5,Palm Beach,FL,33480.0,9874175.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11055042,N,Y,26.719562,-80.034847
23,PAST SALE,2025-03-10,Condo/Co-op,100 SE 5th Ave #101,Boca Raton,FL,33432.0,3475000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11056095,N,Y,26.348292,-80.077094
128,PAST SALE,2025-03-10,Condo/Co-op,2700 N Ocean Dr Unit 304a,Singer Island,FL,33404.0,3375000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-10997456,N,Y,26.785118,-80.033251
87,PAST SALE,2025-03-10,Condo/Co-op,11042 Turtle Beach Rd #202,North Palm Beach,FL,33408.0,3300000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/110...,Beaches MLS,RX-11033561,N,Y,26.837324,-80.042442
3,PAST SALE,2025-03-11,Condo/Co-op,5825 N Ocean Blvd Unit B2,Ocean Ridge,FL,33435.0,3200000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Beaches MLS,RX-11042801,N,Y,26.518048,-80.049758
140,PAST SALE,2025-03-14,Condo/Co-op,429 Australian Ave #3,Palm Beach,FL,33480.0,3125000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Beaches MLS,RX-10944044,N,Y,26.703642,-80.042271
59,PAST SALE,2025-03-14,Condo/Co-op,1802 Captains Way,Jupiter,FL,33477.0,2625000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1802-Captain...,Beaches MLS,RX-11039981,N,Y,26.909756,-80.087596
68,PAST SALE,2025-03-14,Condo/Co-op,2155 S Ocean Blvd #24,Delray Beach,FL,33483.0,2300000.0,4.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Beaches MLS,RX-11043760,N,Y,26.433506,-80.062813
82,PAST SALE,2025-03-11,Condo/Co-op,1800 S Ocean Blvd Ph -C,Boca Raton,FL,33432.0,1815000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1800-S-Oc...,Beaches MLS,RX-11026251,N,Y,26.329315,-80.073835


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [16]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [17]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [18]:
response_df = pd.DataFrame(response_list)

In [19]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [20]:
df_top_ten = merged_df

In [21]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-03-10,Condo/Co-op,1100 S Flagler Dr #1401,West Palm Beach,FL,33401.0,14900000.0,3.0,4.5,...,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Samantha Curry,Douglas Elliman,None,None,Non Agent,Non-Member Selling Office,None,None
1,PAST SALE,2025-03-12,Condo/Co-op,100 Sunrise Ave Ph 5,Palm Beach,FL,33480.0,9874175.0,3.0,3.0,...,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Dana Koch,The Corcoran Group,None,None,Spencer Schlager,Douglas Elliman,None,None
2,PAST SALE,2025-03-10,Condo/Co-op,100 SE 5th Ave #101,Boca Raton,FL,33432.0,3475000.0,3.0,3.0,...,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,James Lawrence,Preferred Residential Prop,None,None
3,PAST SALE,2025-03-10,Condo/Co-op,2700 N Ocean Dr Unit 304a,Singer Island,FL,33404.0,3375000.0,3.0,3.5,...,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,John Nugent,Compass Florida LLC,None,None,Michelle Rinaldi,EXP Realty LLC,None,None
4,PAST SALE,2025-03-10,Condo/Co-op,11042 Turtle Beach Rd #202,North Palm Beach,FL,33408.0,3300000.0,3.0,3.0,...,-80.042442,https://www.redfin.com/FL/North-Palm-Beach/110...,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co
5,PAST SALE,2025-03-11,Condo/Co-op,5825 N Ocean Blvd Unit B2,Ocean Ridge,FL,33435.0,3200000.0,3.0,2.5,...,-80.049758,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Steven Presson,The Corcoran Group,None,None,Steven Presson,The Corcoran Group,John Lipscomb,The Corcoran Group
6,PAST SALE,2025-03-14,Condo/Co-op,429 Australian Ave #3,Palm Beach,FL,33480.0,3125000.0,2.0,2.0,...,-80.042271,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Dana Koch,The Corcoran Group,None,None,Stephen Hall,Compass Florida LLC (PB),None,None
7,PAST SALE,2025-03-14,Condo/Co-op,1802 Captains Way,Jupiter,FL,33477.0,2625000.0,2.0,2.0,...,-80.087596,https://www.redfin.com/FL/Jupiter/1802-Captain...,Thomas Frankel,Admirals Cove Realty Co Inc,Allan Meyerson,Admirals Cove Realty Co Inc,Michelle Navin,Waterfront Properties & Club C,None,None
8,PAST SALE,2025-03-14,Condo/Co-op,2155 S Ocean Blvd #24,Delray Beach,FL,33483.0,2300000.0,4.0,4.0,...,-80.062813,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,Ryan Parker,Coldwell Banker,None,None
9,PAST SALE,2025-03-11,Condo/Co-op,1800 S Ocean Blvd Ph -C,Boca Raton,FL,33432.0,1815000.0,2.0,2.0,...,-80.073835,https://www.redfin.com/FL/Boca-Raton/1800-S-Oc...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Gregory Frank,Compass Florida LLC,None,None


## Current Week's Values

In [22]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

139


In [23]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$693,338


In [24]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$387


In [25]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$96,373,968


In [29]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          139              693338          387                     96373968.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [28]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PDF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
111
-----------
Input Previous Week Condo Average Sales Price:
513901
-----------
Input Previous Week Condo Average PSF:
311
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
57042970.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [30]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [31]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns',None)

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-10,Condo/Co-op,1100 S Flagler Dr #1401,West Palm Beach,FL,33401.0,14900000.0,3.0,4.5,Bristol Condo,4358.0,NaN,2019.0,NaN,3419.0,9368.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-10972146,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Samantha Curry,Douglas Elliman,None,None,Non Agent,Non-Member Selling Office,None,None,1,orange


In [35]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [36]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-10,Condo/Co-op,1100 S Flagler Dr #1401,West Palm Beach,FL,33401.0,14900000.0,3.0,4.5,Bristol Condo,4358.0,NaN,2019.0,NaN,3419.0,9368.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-10972146,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Samantha Curry,Douglas Elliman,None,None,Non Agent,Non-Member Selling Office,None,None,1,orange
1,PAST SALE,2025-03-12,Condo/Co-op,100 Sunrise Ave Ph 5,Palm Beach,FL,33480.0,9874175.0,3.0,3.0,Sun & Surf One Hundred And Thirty Condos,2991.0,NaN,1977.0,NaN,3301.0,4746.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11055042,N,Y,26.719562,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Dana Koch,The Corcoran Group,None,None,Spencer Schlager,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-03-10,Condo/Co-op,100 SE 5th Ave #101,Boca Raton,FL,33432.0,3475000.0,3.0,3.0,Mizner Court,2772.0,NaN,1988.0,NaN,1254.0,3543.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11056095,N,Y,26.348292,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,James Lawrence,Preferred Residential Prop,None,None,3,blue
3,PAST SALE,2025-03-10,Condo/Co-op,2700 N Ocean Dr Unit 304a,Singer Island,FL,33404.0,3375000.0,3.0,3.5,Ritz Carlton Residence,3605.0,NaN,2008.0,NaN,936.0,5341.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-10997456,N,Y,26.785118,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,John Nugent,Compass Florida LLC,None,None,Michelle Rinaldi,EXP Realty LLC,None,None,4,blue
4,PAST SALE,2025-03-10,Condo/Co-op,11042 Turtle Beach Rd #202,North Palm Beach,FL,33408.0,3300000.0,3.0,3.0,Greathouse Condo,2230.0,NaN,1985.0,NaN,1480.0,3558.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/110...,Beaches MLS,RX-11033561,N,Y,26.837324,-80.042442,https://www.redfin.com/FL/North-Palm-Beach/110...,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,5,blue
5,PAST SALE,2025-03-11,Condo/Co-op,5825 N Ocean Blvd Unit B2,Ocean Ridge,FL,33435.0,3200000.0,3.0,2.5,Portofino OF Ocean Ridge Condo,2702.0,NaN,1995.0,NaN,1184.0,3100.0,Sold,NaN,NaN,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Beaches MLS,RX-11042801,N,Y,26.518048,-80.049758,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Steven Presson,The Corcoran Group,None,None,Steven Presson,The Corcoran Group,John Lipscomb,The Corcoran Group,6,blue
6,PAST SALE,2025-03-14,Condo/Co-op,429 Australian Ave #3,Palm Beach,FL,33480.0,3125000.0,2.0,2.0,Australian OF Palm Beach Cond,1714.0,NaN,1974.0,NaN,1823.0,2153.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Beaches MLS,RX-10944044,N,Y,26.703642,-80.042271,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Dana Koch,The Corcoran Group,None,None,Stephen Hall,Compass Florida LLC (PB),None,None,7,blue
7,PAST SALE,2025-03-14,Condo/Co-op,1802 Captains Way,Jupiter,FL,33477.0,2625000.0,2.0,2.0,Admirals Cove,2365.0,NaN,1987.0,NaN,1110.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1802-Captain...,Beaches MLS,RX-11039981,N,Y,26.909756,-80.087596,https://www.redfin.com/FL/Jupiter/1802-Captain...,Thomas Frankel,Admirals Cove Realty Co Inc,Allan Meyerson,Admirals Cove Realty Co Inc,Michelle Navin,Waterfront Properties & Club C,None,None,8,blue
8,PAST SALE,2025-03-14,Condo/Co-op,2155 S

In [37]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [38]:
m.save('index.html')

## Data snagger

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-10,Condo/Co-op,1100 S Flagler Dr #1401,West Palm Beach,FL,33401.0,14900000.0,3.0,4.5,Bristol Condo,4358.0,NaN,2019.0,NaN,3419.0,9368.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-10972146,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Samantha Curry,Douglas Elliman,None,None,Non Agent,Non-Member Selling Office,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_03172024


# CREATE TEMPLATE 

In [42]:
muni_set = set(df_top_ten['CITY'])

In [43]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [44]:
df_top_ten.reset_index(inplace=True,drop=True)

In [45]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [46]:
top_sale

'Bristol Condo at 1100 S Flagler Dr #1401 in West Palm Beach'

In [47]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-10,Condo/Co-op,1100 S Flagler Dr #1401,West Palm Beach,FL,33401.0,14900000.0,3.0,4.5,Bristol Condo,4358.0,NaN,2019.0,NaN,3419.0,9368.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-10972146,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Samantha Curry,Douglas Elliman,None,None,Non Agent,Non-Member Selling Office,None,None,1,orange,Bristol Condo at 1100 S Flagler Dr #1401 in We...
1,PAST SALE,2025-03-12,Condo/Co-op,100 Sunrise Ave Ph 5,Palm Beach,FL,33480.0,9874175.0,3.0,3.0,Sun & Surf One Hundred And Thirty Condos,2991.0,NaN,1977.0,NaN,3301.0,4746.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11055042,N,Y,26.719562,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Dana Koch,The Corcoran Group,None,None,Spencer Schlager,Douglas Elliman,None,None,2,blue,Sun & Surf One Hundred And Thirty Condos at 10...
2,PAST SALE,2025-03-10,Condo/Co-op,100 SE 5th Ave #101,Boca Raton,FL,33432.0,3475000.0,3.0,3.0,Mizner Court,2772.0,NaN,1988.0,NaN,1254.0,3543.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11056095,N,Y,26.348292,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,James Lawrence,Preferred Residential Prop,None,None,3,blue,Mizner Court at 100 SE 5th Ave #101 in Boca Raton
3,PAST SALE,2025-03-10,Condo/Co-op,2700 N Ocean Dr Unit 304a,Singer Island,FL,33404.0,3375000.0,3.0,3.5,Ritz Carlton Residence,3605.0,NaN,2008.0,NaN,936.0,5341.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-10997456,N,Y,26.785118,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,John Nugent,Compass Florida LLC,None,None,Michelle Rinaldi,EXP Realty LLC,None,None,4,blue,Ritz Carlton Residence at 2700 N Ocean Dr Unit...
4,PAST SALE,2025-03-10,Condo/Co-op,11042 Turtle Beach Rd #202,North Palm Beach,FL,33408.0,3300000.0,3.0,3.0,Greathouse Condo,2230.0,NaN,1985.0,NaN,1480.0,3558.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/110...,Beaches MLS,RX-11033561,N,Y,26.837324,-80.042442,https://www.redfin.com/FL/North-Palm-Beach/110...,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,5,blue,Greathouse Condo at 11042 Turtle Beach Rd #202...
5,PAST SALE,2025-03-11,Condo/Co-op,5825 N Ocean Blvd Unit B2,Ocean Ridge,FL,33435.0,3200000.0,3.0,2.5,Portofino OF Ocean Ridge Condo,2702.0,NaN,1995.0,NaN,1184.0,3100.0,Sold,NaN,NaN,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Beaches MLS,RX-11042801,N,Y,26.518048,-80.049758,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Steven Presson,The Corcoran Group,None,None,Steven Presson,The Corcoran Group,John Lipscomb,The Corcoran Group,6,blue,Portofino OF Ocean Ridge Condo at 5825 N Ocean...
6,PAST SALE,2025-03-14,Condo/Co-op,429 Australian Ave #3,Palm Beach,FL,33480.0,3125000.0,2.0,2.0,Australian OF Palm Beach Cond,1714.0,NaN,1974.0,NaN,1823.0,2153.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Beaches MLS,RX-10944044,N,Y,26.703642,-80.042271,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Dana Koch,The Corcoran Group,None,None,Stephen Hall,Compass Florida LLC (PB),None,None,7,blue,Australian OF Palm Beach Cond at 429 Australia...
7,PAST SALE,2025-03-14,Condo/Co-op,1802 Captains Way,Jupiter,FL,33477.0,2625000.0,2.0,2.0,Admirals Cove,2365.0,NaN,1987.0,NaN,1110.0

In [50]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Bristol Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,815,000 to $14,900,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Boca Raton, West Palm Beach, Palm Beach, Singer Island, Ocean Ridge, Jupiter, Delray Beach, North Palm Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 139 condo sales totaling $96,373,968 million from Mar. 9th to Mar. 16th. The previous week, brokers closed 111 condo sales totaling $57,042,970.

Last 

In [51]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [52]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-10,Condo/Co-op,1100 S Flagler Dr #1401,West Palm Beach,FL,33401.0,14900000.0,3.0,4.5,Bristol Condo,4358.0,NaN,2019.0,NaN,3419.0,9368.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-10972146,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Samantha Curry,Douglas Elliman,None,None,Non Agent,Non-Member Selling Office,None,None,1,orange,Bristol Condo at 1100 S Flagler Dr #1401 in We...
1,PAST SALE,2025-03-12,Condo/Co-op,100 Sunrise Ave Ph 5,Palm Beach,FL,33480.0,9874175.0,3.0,3.0,Sun & Surf One Hundred And Thirty Condos,2991.0,NaN,1977.0,NaN,3301.0,4746.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11055042,N,Y,26.719562,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Dana Koch,The Corcoran Group,None,None,Spencer Schlager,Douglas Elliman,None,None,2,blue,Sun & Surf One Hundred And Thirty Condos at 10...
2,PAST SALE,2025-03-10,Condo/Co-op,100 SE 5th Ave #101,Boca Raton,FL,33432.0,3475000.0,3.0,3.0,Mizner Court,2772.0,NaN,1988.0,NaN,1254.0,3543.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11056095,N,Y,26.348292,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,James Lawrence,Preferred Residential Prop,None,None,3,blue,Mizner Court at 100 SE 5th Ave #101 in Boca Raton
3,PAST SALE,2025-03-10,Condo/Co-op,2700 N Ocean Dr Unit 304a,Singer Island,FL,33404.0,3375000.0,3.0,3.5,Ritz Carlton Residence,3605.0,NaN,2008.0,NaN,936.0,5341.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-10997456,N,Y,26.785118,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,John Nugent,Compass Florida LLC,None,None,Michelle Rinaldi,EXP Realty LLC,None,None,4,blue,Ritz Carlton Residence at 2700 N Ocean Dr Unit...
4,PAST SALE,2025-03-10,Condo/Co-op,11042 Turtle Beach Rd #202,North Palm Beach,FL,33408.0,3300000.0,3.0,3.0,Greathouse Condo,2230.0,NaN,1985.0,NaN,1480.0,3558.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/110...,Beaches MLS,RX-11033561,N,Y,26.837324,-80.042442,https://www.redfin.com/FL/North-Palm-Beach/110...,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,5,blue,Greathouse Condo at 11042 Turtle Beach Rd #202...
5,PAST SALE,2025-03-11,Condo/Co-op,5825 N Ocean Blvd Unit B2,Ocean Ridge,FL,33435.0,3200000.0,3.0,2.5,Portofino OF Ocean Ridge Condo,2702.0,NaN,1995.0,NaN,1184.0,3100.0,Sold,NaN,NaN,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Beaches MLS,RX-11042801,N,Y,26.518048,-80.049758,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Steven Presson,The Corcoran Group,None,None,Steven Presson,The Corcoran Group,John Lipscomb,The Corcoran Group,6,blue,Portofino OF Ocean Ridge Condo at 5825 N Ocean...
6,PAST SALE,2025-03-14,Condo/Co-op,429 Australian Ave #3,Palm Beach,FL,33480.0,3125000.0,2.0,2.0,Australian OF Palm Beach Cond,1714.0,NaN,1974.0,NaN,1823.0,2153.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Beaches MLS,RX-10944044,N,Y,26.703642,-80.042271,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Dana Koch,The Corcoran Group,None,None,Stephen Hall,Compass Florida LLC (PB),None,None,7,blue,Australian OF Palm Beach Cond at 429 Australia...
7,PAST SALE,2025-03-14,Condo/Co-op,1802 Captains Way,Jupiter,FL,33477.0,2625000.0,2.0,2.0,Admirals Cove,2365.0,NaN,1987.0,NaN,1110.0

In [55]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Bristol Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,815,000 to $14,900,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Boca Raton, West Palm Beach, Palm Beach, Singer Island, Ocean Ridge, Jupiter, Delray Beach, North Palm Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 139 condo sales totaling $96,373,968 million from Mar. 9th to Mar. 16th. The previous week, brokers closed 111 condo sales totaling $57,042,970.

Last 

In [56]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/West-Palm-Beach/1100-S-Flagler-Dr-33401/unit-1401/home/168183435


In [57]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/100-Sunrise-Ave-33480/unit-5/home/42437792


In [58]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Boca-Raton/1800-S-Ocean-Blvd-33432/unit-0C/home/79743386


In [59]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Delray-Beach/2155-S-Ocean-Blvd-33483/unit-24/home/42574061


In [60]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/West-Palm-Beach/1100-S-Flagler-Dr-33401/unit-1401/home/168183435


In [61]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Delray-Beach/2155-S-Ocean-Blvd-33483/unit-24/home/42574061


In [62]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-10,Condo/Co-op,1100 S Flagler Dr #1401,West Palm Beach,FL,33401.0,14900000.0,3.0,4.5,Bristol Condo,4358.0,NaN,2019.0,NaN,3419.0,9368.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-10972146,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Samantha Curry,Douglas Elliman,None,None,Non Agent,Non-Member Selling Office,None,None,1,orange,Bristol Condo at 1100 S Flagler Dr #1401 in We...
1,PAST SALE,2025-03-12,Condo/Co-op,100 Sunrise Ave Ph 5,Palm Beach,FL,33480.0,9874175.0,3.0,3.0,Sun & Surf One Hundred And Thirty Condos,2991.0,NaN,1977.0,NaN,3301.0,4746.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11055042,N,Y,26.719562,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Dana Koch,The Corcoran Group,None,None,Spencer Schlager,Douglas Elliman,None,None,2,blue,Sun & Surf One Hundred And Thirty Condos at 10...
2,PAST SALE,2025-03-10,Condo/Co-op,100 SE 5th Ave #101,Boca Raton,FL,33432.0,3475000.0,3.0,3.0,Mizner Court,2772.0,NaN,1988.0,NaN,1254.0,3543.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11056095,N,Y,26.348292,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,James Lawrence,Preferred Residential Prop,None,None,3,blue,Mizner Court at 100 SE 5th Ave #101 in Boca Raton
3,PAST SALE,2025-03-10,Condo/Co-op,2700 N Ocean Dr Unit 304a,Singer Island,FL,33404.0,3375000.0,3.0,3.5,Ritz Carlton Residence,3605.0,NaN,2008.0,NaN,936.0,5341.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-10997456,N,Y,26.785118,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,John Nugent,Compass Florida LLC,None,None,Michelle Rinaldi,EXP Realty LLC,None,None,4,blue,Ritz Carlton Residence at 2700 N Ocean Dr Unit...
4,PAST SALE,2025-03-10,Condo/Co-op,11042 Turtle Beach Rd #202,North Palm Beach,FL,33408.0,3300000.0,3.0,3.0,Greathouse Condo,2230.0,NaN,1985.0,NaN,1480.0,3558.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/110...,Beaches MLS,RX-11033561,N,Y,26.837324,-80.042442,https://www.redfin.com/FL/North-Palm-Beach/110...,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,Jeffrey Hallas,Illustrated Properties LLC (Co,John Sousa,Illustrated Properties LLC (Co,5,blue,Greathouse Condo at 11042 Turtle Beach Rd #202...
5,PAST SALE,2025-03-11,Condo/Co-op,5825 N Ocean Blvd Unit B2,Ocean Ridge,FL,33435.0,3200000.0,3.0,2.5,Portofino OF Ocean Ridge Condo,2702.0,NaN,1995.0,NaN,1184.0,3100.0,Sold,NaN,NaN,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Beaches MLS,RX-11042801,N,Y,26.518048,-80.049758,https://www.redfin.com/FL/Ocean-Ridge/5825-N-O...,Steven Presson,The Corcoran Group,None,None,Steven Presson,The Corcoran Group,John Lipscomb,The Corcoran Group,6,blue,Portofino OF Ocean Ridge Condo at 5825 N Ocean...
6,PAST SALE,2025-03-14,Condo/Co-op,429 Australian Ave #3,Palm Beach,FL,33480.0,3125000.0,2.0,2.0,Australian OF Palm Beach Cond,1714.0,NaN,1974.0,NaN,1823.0,2153.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Beaches MLS,RX-10944044,N,Y,26.703642,-80.042271,https://www.redfin.com/FL/Palm-Beach/429-Austr...,Dana Koch,The Corcoran Group,None,None,Stephen Hall,Compass Florida LLC (PB),None,None,7,blue,Australian OF Palm Beach Cond at 429 Australia...
7,PAST SALE,2025-03-14,Condo/Co-op,1802 Captains Way,Jupiter,FL,33477.0,2625000.0,2.0,2.0,Admirals Cove,2365.0,NaN,1987.0,NaN,1110.0

## Time on Market Calculator

In [64]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 12) ## List (Earlier) date
date2 = datetime(2025, 3, 12) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

181


## Clean CSV for Datawrapper Chart

In [65]:
chart_df = df_top_ten

In [66]:
chart_df = chart_df.fillna(" ")

In [67]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [68]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [69]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [70]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [71]:
chart_df['$/SQUARE FEET'].astype(float)

0    3419.0
1    3301.0
2    1254.0
3     936.0
4    1480.0
5    1184.0
6    1823.0
7    1110.0
8     695.0
9    1027.0
Name: $/SQUARE FEET, dtype: float64

In [72]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [73]:
csv_date_string = today.strftime("%m_%d_%Y")

In [74]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [75]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-03-10,1100 S Flagler Dr #1401 in West Palm Beach,"$14,900,000",3,4.5,"4,358",2019,"$3,419",Beaches MLS,RX-10972146,Samantha Curry Douglas Elliman,Non Agent Non-Member Selling Office
1,2025-03-12,100 Sunrise Ave Ph 5 in Palm Beach,"$9,874,175",3,3.0,"2,991",1977,"$3,301",Beaches MLS,RX-11055042,Dana Koch The Corcoran Group,Spencer Schlager Douglas Elliman
2,2025-03-10,100 SE 5th Ave #101 in Boca Raton,"$3,475,000",3,3.0,"2,772",1988,"$1,254",Beaches MLS,RX-11056095,David Hall Compass Florida LLC Sonia Hall Comp...,James Lawrence Preferred Residential Prop
3,2025-03-10,2700 N Ocean Dr Unit 304a in Singer Island,"$3,375,000",3,3.5,"3,605",2008,$936,Beaches MLS,RX-10997456,John Nugent Compass Florida LLC,Michelle Rinaldi EXP Realty LLC
4,2025-03-10,11042 Turtle Beach Rd #202 in North Palm Beach,"$3,300,000",3,3.0,"2,230",1985,"$1,480",Beaches MLS,RX-11033561,Jeffrey Hallas Illustrated Properties LLC (Co ...,Jeffrey Hallas Illustrated Properties LLC (Co ...
5,2025-03-11,5825 N Ocean Blvd Unit B2 in Ocean Ridge,"$3,200,000",3,2.5,"2,702",1995,"$1,184",Beaches MLS,RX-11042801,Steven Presson The Corcoran Group,Steven Presson The Corcoran Group John Lipscom...
6,2025-03-14,429 Australian Ave #3 in Palm Beach,"$3,125,000",2,2.0,"1,714",1974,"$1,823",Beaches MLS,RX-10944044,Dana Koch The Corcoran Group,Stephen Hall Compass Florida LLC (PB)
7,2025-03-14,1802 Captains Way in Jupiter,"$2,625,000",2,2.0,"2,365",1987,"$1,110",Beaches MLS,RX-11039981,Thomas Frankel Admirals Cove Realty Co Inc All...,Michelle Navin Waterfront Properties & Club C
8,2025-03-14,2155 S Ocean Blvd #24 in Delray Beach,"$2,300,000",4,4.0,"3,308",1981,$695,Beaches MLS,RX-11043760,Candace Friis The Corcoran Group Phillip Friis...,Ryan Parker Coldwell Banker
9,2025-03-11,1800 S Ocean Blvd Ph -C in Boca Raton,"$1,815,000",2,2.0,"1,768",1979,"$1,027",Beaches MLS,RX-11026251,Michael Wells Premier Estate Properties Inc Sc...,Gregory Frank Compass Florida LLC
